In [7]:
from keras.models import Sequential, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers.crf import CRF
from keras_contrib.utils import save_load_utils
from keras_contrib.metrics import crf_accuracy
from keras_contrib.losses import crf_loss
from keras.utils import to_categorical
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class LSTMmodel:
    def __init__(self, input_length, para_emb_dim, num_tags, hidden_dim=200, dropout=0.5):
        self.num_tags = num_tags
        self.model = Sequential()
        self.model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True), input_shape=(input_length, para_emb_dim)))
        self.model.add(Dropout(dropout))
        # self.model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True), input_shape=(input_length, para_emb_dim)))
        # self.model.add(Dropout(dropout))
        self.model.add(TimeDistributed(Dense(self.num_tags)))
        crf = CRF(self.num_tags)
        self.model.add(crf)
        self.model.compile('rmsprop', loss=losses.crf_loss, metrics=[crf_accuracy])
    
    def save_model(self, filepath):
        save_load_utils.save_all_weights(self.model, filepath)
    
    def restore_model(self, filepath):
        save_load_utils.load_all_weights(self.model, filepath)
        
    def train(self, trainX, trainY, batch_size=32, epochs=10, validation_split=0.1, verbose=1):
        return self.model.fit(trainX, np.array(trainY), batch_size=batch_size, epochs=epochs, 
                             validation_split=validation_split, verbose=verbose)
        

In [ ]:
id2label = {0: 'PAD', 1: 'B0', 2: 'B1', 3: 'B2', 4: 'B3', 5: 'B4', 6: 'B-1', 7: 'B-2', 8: 'B-3', 9: 'B-4', 10: 'M', 11: 'E'}
label2id = {}
for i in id2label:
    label2id[id2label[i]] = i

In [ ]:
INPUT_LENGTH = 270
PARAGRAPH_EMB_DIM = 300
NUM_TAGS = 12

model = LSTMmodel(INPUT_LENGTH, PARAGRAPH_EMB_DIM, NUM_TAGS)
model.model.summary()

In [ ]:
# load data
X, rawY = [], [] # X is 3D: article, paragraph, embedding; Y is 2D: article, paragraph
Y = [to_categorical(y, num_classes=NUM_TAGS) for y in rawY] # Y is now 3D
data_size = len(X)
train_size = int(data_size * 0.9)
trainX, trainY = X[:train_size], Y[:train_size]
testX, testY = X[train_size:], Y[train_size:]

In [ ]:
# train
history = model.model.train(trainX, trainY, batch_size=32, epochs=10)

In [ ]:
# plot
# plt.style.use("ggplot")
# plt.figure(figsize=(12,12))
# plt.plot(hist["acc"])
# plt.plot(hist["val_acc"])
# plt.show()

In [ ]:
# Predict on test
test_pred = model.predict(testX, verbose=1)